In [59]:
import random
import json 
import pickle
import numpy as np
import json

import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/test/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [33]:
lemmatizer = WordNetLemmatizer()
intents =json.loads(open("intents.json").read())

words = []
classes = []
documents = []
ignore_letters = ["?", "!", ".",","]

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent["tag"]))
    if intent["tag"] not in classes:
      classes.append(intent["tag"])
  

In [34]:
words = sorted(set([lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]))

In [35]:
classes = sorted(set(classes))
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(words, open("classes.pkl","wb"))

In [56]:
training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  words_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  
  [bag.append(1) if word in words_patterns else bag.append(0) for word in words]
  
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag,output_row])


random.shuffle(training)
training = np.array(training,dtype=object)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [61]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation ="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation ="softmax"))

sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6,momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

model.save("chatbot_model.h5", hist)
print("Model has been completed.")

/Users/test/miniforge3/envs/machine-learning-env/lib/python3.11/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 0s 1ms/step - loss: 1.6697 - accuracy: 0.1724
Epoch 2/200
6/6 [==============================] - 0s 752us/step - loss: 1.5202 - accuracy: 0.4138
Epoch 3/200
6/6 [==============================] - 0s 848us/step - loss: 1.5802 - accuracy: 0.3448
Epoch 4/200
6/6 [==============================] - 0s 702us/step - loss: 1.5371 - accuracy: 0.3103
Epoch 5/200
6/6 [==============================] - 0s 746us/step - loss: 1.4558 - accuracy: 0.4483
Epoch 6/200
6/6 [==============================] - 0s 968us/step - loss: 1.4187 - accuracy: 0.4828
Epoch 7/200
6/6 [==============================] - 0s 978us/step - loss: 1.2859 - accuracy: 0.5862
Epoch 8/200
6/6 [==============================] - 0s 1ms/step - loss: 1.3249 - accuracy: 0.5517
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.2042 - accuracy: 0.7931
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.0851 - accuracy: 0.7586
Epoch 11/200
6/6 

/Users/test/miniforge3/envs/machine-learning-env/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [60]:

words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))
model = load_model("chatbot_model.model")

def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  return [lemmatizer.lemmatize(word) for word in sentence_words]

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  
  for block in sentence_words:
    for idx, word in enumerate(words):
      if word == block:
        bag[idx] =1
  
  return np.array(bag)

def predict_class(sentence):
  bow= bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRES = 0.25
  result = [[i,r] for i,r in enumerate(res) if r > ERROR_THRES]
  
  result.sort(key=lambda x:x[1], reverse=True)
  return_list = []
  for r in result:
    return_list.append({"intent":classes[r[0]], "probability": str(r[1])})
  
  return return_list